# Googleドライブから学習データをロード

In [ ]:
from google.colab import drive
drive.mount('./gdrive')

In [2]:
!cp '/content/gdrive/MyDrive/ReversiTrainData/X_dataset2.npy' ./
!cp '/content/gdrive/MyDrive/ReversiTrainData/y_dataset2.npy' ./

In [3]:
import numpy as np
X_dataset = np.load('X_dataset2.npy')
y_dataset = np.load('y_dataset2.npy')
print(len(X_dataset))

4011482


In [4]:
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.8, random_state=RANDOM_SEED)

# モデル構築

In [5]:
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard

In [6]:
input_size = len(X_dataset[0])
print(input_size)

64


In [7]:
NUM_CLASSES = len(y_dataset[0])
print(NUM_CLASSES)

64


In [8]:
model_save_path = 'reversi_ai.hdf5'

In [9]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((input_size, )),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(96, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               16640     
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 96)                12384     
_________________________________________________________________
dense_3 (Dense)              (None, 64)                6208      
Total params: 68,128
Trainable params: 68,128
Non-trainable params: 0
_________________________________________________________________


In [11]:
# モデルチェックポイントのコールバック
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# 早期打ち切り用コールバック
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [12]:
# モデルコンパイル
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 学習

In [13]:
epochs = 1000
batch_size = 128

In [14]:
model.fit(
    X_train,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
25072/25072 [==============================] - 75s 3ms/step - loss: 2.3938 - accuracy: 0.2775 - val_loss: 2.0747 - val_accuracy: 0.3317

Epoch 00001: saving model to reversi_ai.hdf5
Epoch 2/1000
25072/25072 [==============================] - 71s 3ms/step - loss: 2.0262 - accuracy: 0.3418 - val_loss: 1.9596 - val_accuracy: 0.3543

Epoch 00002: saving model to reversi_ai.hdf5
Epoch 3/1000
25072/25072 [==============================] - 72s 3ms/step - loss: 1.9262 - accuracy: 0.3644 - val_loss: 1.9008 - val_accuracy: 0.3733

Epoch 00003: saving model to reversi_ai.hdf5
Epoch 4/1000
25072/25072 [==============================] - 72s 3ms/step - loss: 1.8606 - accuracy: 0.3808 - val_loss: 1.8161 - val_accuracy: 0.3902

Epoch 00004: saving model to reversi_ai.hdf5
Epoch 5/1000
25072/25072 [==============================] - 72s 3ms/step - loss: 1.8075 - accuracy: 0.3944 - val_loss: 1.7831 - val_accuracy: 0.3973

Epoch 00005: saving model to reversi_ai.hdf5
Epoch 6/1000
25072/25072 

In [15]:
# モデルコンパイル
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [16]:
model.fit(
    X_train,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
25072/25072 [==============================] - 72s 3ms/step - loss: 1.4742 - accuracy: 0.4777 - val_loss: 1.4783 - val_accuracy: 0.4772

Epoch 00001: saving model to reversi_ai.hdf5
Epoch 2/1000
25072/25072 [==============================] - 72s 3ms/step - loss: 1.4512 - accuracy: 0.4835 - val_loss: 1.4750 - val_accuracy: 0.4787

Epoch 00002: saving model to reversi_ai.hdf5
Epoch 3/1000
25072/25072 [==============================] - 72s 3ms/step - loss: 1.4465 - accuracy: 0.4847 - val_loss: 1.4720 - val_accuracy: 0.4788

Epoch 00003: saving model to reversi_ai.hdf5
Epoch 4/1000
25072/25072 [==============================] - 72s 3ms/step - loss: 1.4434 - accuracy: 0.4859 - val_loss: 1.4700 - val_accuracy: 0.4798

Epoch 00004: saving model to reversi_ai.hdf5
Epoch 5/1000
25072/25072 [==============================] - 72s 3ms/step - loss: 1.4421 - accuracy: 0.4858 - val_loss: 1.4708 - val_accuracy: 0.4791

Epoch 00005: saving model to reversi_ai.hdf5
Epoch 6/1000
25072/25072 

In [17]:
# 保存したモデルのロード
model = tf.keras.models.load_model(model_save_path)

In [18]:
# 推論テスト
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

[2.45732474e-15 6.85066709e-07 1.36058361e-07 1.72398515e-07
 1.18667795e-05 1.28313715e-08 6.29287982e-08 2.86560087e-09
 1.26416957e-08 1.75602474e-06 4.53798585e-02 4.49770050e-06
 4.37122027e-09 1.00344881e-01 3.43067659e-05 1.32245838e-03
 2.66032264e-04 1.25176394e-02 2.57371283e-07 8.09562150e-21
 7.60068922e-15 1.26909677e-10 7.72949022e-07 2.38819607e-02
 1.84778664e-02 2.01226547e-04 1.34229730e-08 0.00000000e+00
 0.00000000e+00 8.89600713e-08 2.61334844e-05 8.24281722e-02
 9.81501769e-03 5.21296761e-06 2.93508266e-14 0.00000000e+00
 0.00000000e+00 3.97367387e-07 7.46153637e-06 1.14659108e-01
 2.19979438e-06 3.62994187e-02 2.33793821e-06 1.03182899e-07
 7.71572886e-08 2.93413937e-01 1.97541162e-01 9.78684014e-08
 1.41675503e-06 2.37154272e-05 1.24666011e-02 2.74754949e-02
 2.32406240e-02 3.86355896e-06 2.61916684e-05 9.45496577e-05
 1.82393232e-11 1.22542588e-06 8.05790592e-07 3.12213544e-09
 1.79895142e-05 3.65452937e-12 9.16013860e-08 2.58152095e-14]
45


# ONNXへ変換

In [19]:
# モデル保存
model.save('saved_model', save_format='tf')

INFO:tensorflow:Assets written to: saved_model/assets


In [20]:
!pip install onnxruntime
!pip install tf2onnx

     |████████████████████████████████| 4.1MB 16.6MB/s 
     |████████████████████████████████| 337kB 23.3MB/s 
     |████████████████████████████████| 14.5MB 53.4MB/s 


In [21]:
# ONNXへ変換
!python -m tf2onnx.convert --saved-model='saved_model' \
                           --opset=9 \
                           --output='reversi_ai.onnx'

2021-03-07 22:37:26.325610: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-03-07 22:37:29,342 - WARNING - '--tag' not specified for saved_model. Using --tag serve
2021-03-07 22:37:29.354265: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-03-07 22:37:29.359106: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-03-07 22:37:29.364735: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-03-07 22:37:29.365292: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 29

In [22]:
!cp 'reversi_ai.onnx' '/content/gdrive/MyDrive/ReversiTrainData'